In [1]:
from tkinter import Menu
from matplotlib.style import use
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
import telegram
import logging
import sqlite3
from telegram.ext import (
    Updater,
    CallbackQueryHandler,
    CommandHandler,
    ConversationHandler,
    CallbackContext, 
    MessageHandler, 
    Filters
    )

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

updater = Updater(token='1686756740:AAGwTrm-o91o17n3-DwkTUxJ2Mnrvwy1kCE')
dispatcher = updater.dispatcher
job_queue = updater.job_queue

STEP_ONE, STEP_TWO, STEP_THREE, STEP_FOUR = range(4)

user_data = dict()

def start(update: Update, context: CallbackContext) -> None:
    menu(update, context)

#Main menu function (Summoned by player)
def menu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Start Game", callback_data="1")
        ],
        [
            InlineKeyboardButton("Options", callback_data="2")
        ],
        [
            InlineKeyboardButton("Credits", callback_data="3")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("""Kingdom of Death:
        Dancing Dolls
    
    Main Menu:""", reply_markup=reply_markup)

#Main menu function (automatic)
def amenu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Start Game", callback_data="1")
        ],
        [
            InlineKeyboardButton("Options", callback_data="2")
        ],
        [
            InlineKeyboardButton("Credits", callback_data="3")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("""Kingdom of Death:
        Dancing Dolls
    
    Main Menu:""", reply_markup=reply_markup)

#Start Game
def start_game(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("New Game", callback_data="5")
        ],
        [
            InlineKeyboardButton("Load Game", callback_data="6")
        ],
        [
            InlineKeyboardButton("Back", callback_data="4")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose:", reply_markup=reply_markup)

#New Game
    #Name
def user_name(update: Update, context:CallbackContext) -> None:
    chat = update.effective_chat
    context.bot.send_message(chat_id=chat.id, text="Hero, what is your name?")
    if update.message.text:
        user_sex(update, context)
    #Sex
def user_sex(update: Update, context:CallbackContext) -> None:
    chat = update.effective_chat
    user_data['name'] = update.message.text

    keyboard = [
        [
            InlineKeyboardButton("Hero (Male)", callback_data="10")
        ],
        [
            InlineKeyboardButton("Heroine(Female)", callback_data="11")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Are you Hero or Heroine?", reply_markup=reply_markup)
    #Race
def user_race(update: Update, context:CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Human", callback_data="12")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("What are you?", reply_markup=reply_markup)
    
    #Starter Items
def starter_pack(update: Update, context:CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("""I were a knight...  (Heavy armory and equipment)""", callback_data="13")
        ],
        [
            InlineKeyboardButton("""I were a freelancer...  (Light leather armor and early real arms)""", callback_data="14")
        ],
        [
            InlineKeyboardButton("""I were an adventurer    (You have Nothing, but small axe)""", callback_data="15")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Whom were you?", reply_markup=reply_markup)

#Credits
def back_main_menu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Back", callback_data="4")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Back to main menu?", reply_markup=reply_markup)

#buttons reader
def button(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()

    if query.data == "1":
        query.edit_message_text(text=f"The Bridge behind fog...")
        start_game(update, context)
    elif query.data == "2":
        query.edit_message_text(text=f"Here are no options... by now...")
        back_main_menu(update, context)
    elif query.data == "3":
        query.edit_message_text(text=f"Game made by PupetOne")
        back_main_menu(update, context)
    elif query.data == "4":
        query.edit_message_text(text=f"Gears tick back in time...")
        amenu(update, context)
    elif query.data == "5":
        query.edit_message_text(text=f"Memories flow in river...")
        user_name(update, context)
    elif query.data == "6":
        query.edit_message_text(text=f"WIP... please wait...")    
    elif query.data == "10":
        query.edit_message_text(text=f"Alright, nice to meet you, sir...")
        user_data['sex'] = 'Male' 
        user_race(update, context)
    elif query.data == "11":
        query.edit_message_text(text=f"I'm glad to know you, lady...")   
        user_data['sex'] = 'Female'
        user_race(update, context)
    elif query.data == "12":
        query.edit_message_text(text=f'Interesting... Could you describe us meaning of word "Humanity"?')  
        user_data['race'] = 'Human'
        starter_pack(update, context)
    elif query.data == "13":
        query.edit_message_text(text=f"So, {user_data['name']}, you had enough money for preparations, how I see...")
    elif query.data == "14":
        query.edit_message_text(text=f"{user_data['name']}, free as birds, yes?")
    elif query.data == "15":
        query.edit_message_text(text=f"{user_data['name']}, it's never late to begin.")
    #elif query.data == "":
        #query.edit_message_text(text=f"")
    
    

#def echo(update: Update, context: CallbackContext):
#    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

def stop(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Shutting down...")
    updater.stop()

def start_callback(context: telegram.ext.CallbackContext):
    context.bot.send_message(chat_id='720434978', text="""Golem was awaken...
    
    *Type '/start' to begin*""")

def player_reply(update, context):
    user_says = ''.join(context.args)
    update.message.reply_text(f"You said: {user_says}")

job_minute = job_queue.run_once(start_callback, 1)

#Start bot
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

#Main menu
menu_handler = CommandHandler("menu", menu)
dispatcher.add_handler(menu_handler)

amenu_handler = CommandHandler("amenu", amenu)
dispatcher.add_handler(amenu_handler)

start_game_handler = CommandHandler("start_game", start_game)
dispatcher.add_handler(start_game_handler)

#User name
user_name_handler = MessageHandler(Filters.text & (~ Filters.command), user_sex)
dispatcher.add_handler(user_name_handler)
#User sex
user_sex_handler = CommandHandler("user_sex", user_sex)
dispatcher.add_handler(user_sex_handler)
#User race
user_race_handler = CommandHandler("user_race", user_race)
dispatcher.add_handler(user_race_handler)
#User starter pack
user_spack_handler = CommandHandler("user_spack", starter_pack)
dispatcher.add_handler(user_spack_handler)

back_main_menu_handler = CommandHandler("menu", back_main_menu)
dispatcher.add_handler(back_main_menu_handler)
#Buttons
dispatcher.add_handler(CallbackQueryHandler(button))

#echo
#echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
#dispatcher.add_handler(echo_handler)

#Turn off bot
stop_handler = CommandHandler('stop', stop)
dispatcher.add_handler(stop_handler)

#User info

dispatcher.add_handler(CommandHandler("say", player_reply))

updater.start_polling()
updater.idle()

2022-09-03 11:53:35,604 - apscheduler.scheduler - INFO - Adding job tentatively -- it will be properly scheduled when the scheduler starts
2022-09-03 11:53:35,606 - apscheduler.scheduler - INFO - Added job "start_callback" to job store "default"
2022-09-03 11:53:35,607 - apscheduler.scheduler - INFO - Scheduler started
2022-09-03 11:53:36,517 - apscheduler.executors.default - INFO - Running job "start_callback (trigger: date[2022-09-03 08:53:36 UTC], next run at: 2022-09-03 08:53:36 UTC)" (scheduled at 2022-09-03 08:53:36.480698+00:00)
2022-09-03 11:53:36,521 - apscheduler.scheduler - INFO - Removed job 7cdc9c4426974e8b87fc539f70abd726
2022-09-03 11:53:36,717 - apscheduler.executors.default - INFO - Job "start_callback (trigger: date[2022-09-03 08:53:36 UTC], next run at: 2022-09-03 08:53:36 UTC)" executed successfully


In [ ]:
from tkinter import Menu
from matplotlib.style import use
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
import telegram
import logging
import sqlite3
from telegram.ext import (
    Updater,
    CallbackQueryHandler,
    CommandHandler,
    ConversationHandler,
    CallbackContext, 
    MessageHandler, 
    Filters
    )

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

updater = Updater(token='1686756740:AAGwTrm-o91o17n3-DwkTUxJ2Mnrvwy1kCE')
dispatcher = updater.dispatcher
job_queue = updater.job_queue

STEP_ONE, STEP_TWO, STEP_THREE, STEP_FOUR = range(4)

user_data = dict()

def start(update: Update, context: CallbackContext) -> None:
    menu(update, context)

#Main menu function (Summoned by player)
def menu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Start Game", callback_data="startGame")
        ],
        [
            InlineKeyboardButton("Options", callback_data="options")
        ],
        [
            InlineKeyboardButton("Credits", callback_data="credits")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("""Kingdom of Death:
        Dancing Dolls
    
    Main Menu:""", reply_markup=reply_markup)

#Main menu function (automatic)
def amenu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Start Game", callback_data="startGame")
        ],
        [
            InlineKeyboardButton("Options", callback_data="options")
        ],
        [
            InlineKeyboardButton("Credits", callback_data="credits")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("""Kingdom of Death:
        Dancing Dolls
    
    Main Menu:""", reply_markup=reply_markup)

#Start Game
def start_game(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("New Game", callback_data="newGame")
        ],
        [
            InlineKeyboardButton("Load Game", callback_data="loadGame")
        ],
        [
            InlineKeyboardButton("Back", callback_data="backMainMenu")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose:", reply_markup=reply_markup)

#New Game
    #Name
def user_name(update: Update, context:CallbackContext):
    chat = update.effective_chat
    context.bot.send_message(chat_id=chat.id, text="Hero, what is your name?")
    
    #if update.message.text:
    #    user_sex(update, context)
    #Sex
def user_sex(update: Update, context:CallbackContext):
    chat = update.effective_chat
    user_data['name'] = update.message.text

    keyboard = [
        [
            InlineKeyboardButton("Hero (Male)", callback_data="10")
        ],
        [
            InlineKeyboardButton("Heroine(Female)", callback_data="11")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Are you Hero or Heroine?", reply_markup=reply_markup)

    return STEP_TWO
    #Race
def user_race(update: Update, context:CallbackContext):
    chat = update.effective_chat
    user_data['sex'] = update.callback_query.message.text
    keyboard = [
        [
            InlineKeyboardButton("Human", callback_data="12")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("What are you?", reply_markup=reply_markup)

    return STEP_THREE

    #Starter Items
def starter_pack(update: Update, context:CallbackContext):
    chat = update.effective_chat
    user_data['race'] = update.callback_query.message.text
    keyboard = [
        [
            InlineKeyboardButton("""I were a knight...  (Heavy armory and equipment)""", callback_data="knightStart")
        ],
        [
            InlineKeyboardButton("""I were a freelancer...  (Light leather armor and early real arms)""", callback_data="freelancerStart")
        ],
        [
            InlineKeyboardButton("""I were an adventurer    (You have Nothing, but small axe)""", callback_data="adventurerStart")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Whom were you?", reply_markup=reply_markup)
    
    return STEP_FOUR

    #Finish registration
def user_finish(update, context):
    chat = update.effective_chat
    user_data['sset'] = update.callback_query.message.text

    user_text = f"Thanks  {user_data.get('name')}"

    context.bot.send_message(chat_id=chat.id, text=user_text)
    return ConversationHandler.END  
#Credits
def back_main_menu(update: Update, context: CallbackContext):
    keyboard = [
        [
            InlineKeyboardButton("Back", callback_data="backMainMenu")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Back to main menu?", reply_markup=reply_markup)
    return ConversationHandler.END  

#buttons reader
def button(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()

    if query.data == "startGame":
        query.edit_message_text(text=f"The Bridge behind fog...")
        start_game(update, context)
    elif query.data == "options":
        query.edit_message_text(text=f"Here are no options... by now...")
        back_main_menu(update, context)
    elif query.data == "credits":
        query.edit_message_text(text=f"Game made by PupetOne")
        back_main_menu(update, context)
    elif query.data == "backMainMenu":
        query.edit_message_text(text=f"Gears tick back in time...")
        amenu(update, context)
    #elif query.data == "":
        #query.edit_message_text(text=f"")
    
    

#def echo(update: Update, context: CallbackContext):
#    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

def stop(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Shutting down...")
    updater.stop()

def start_callback(context: telegram.ext.CallbackContext):
    context.bot.send_message(chat_id='720434978', text="""Golem was awaken...
    
    *Type '/start' to begin*""")

def player_reply(update, context):
    user_says = ''.join(context.args)
    update.message.reply_text(f"You said: {user_says}")

job_minute = job_queue.run_once(start_callback, 1)

#Start bot
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

#Main menu
menu_handler = CommandHandler("menu", menu)
dispatcher.add_handler(menu_handler)

#Registration
def regUser() -> None:

    def button(update: Update, context: CallbackContext) -> None:
        query = update.callback_query
        query.answer()

        if query.data == "newGame":
            query.edit_message_text(text=f"Memories flow in river...")
            user_name(update, context)
        elif query.data == "loadGame":
            query.edit_message_text(text=f"WIP... please wait...")    
        elif query.data == "male":
            query.edit_message_text(text=f"Alright, nice to meet you, sir...")
            #user_data['sex'] = 'Male' 
            user_race(update, context)
        elif query.data == "female":
            query.edit_message_text(text=f"I'm glad to know you, lady...")   
            #user_data['sex'] = 'Female'
            user_race(update, context)
        elif query.data == "humanRace":
            query.edit_message_text(text=f'Interesting... Could you describe us meaning of word "Humanity"?')  
            #user_data['race'] = 'Human'
            starter_pack(update, context)
        elif query.data == "knightStart":
            query.edit_message_text(text=f"So, {user_data['name']}, you had enough money for preparations, how I see...")
        elif query.data == "freelancerStart":
            query.edit_message_text(text=f"{user_data['name']}, free as birds, yes?")
        elif query.data == "adventurerStart":
            query.edit_message_text(text=f"{user_data['name']}, it's never late to begin.")

    #User name
    user_name_handler = MessageHandler(Filters.text & (~ Filters.command), user_sex)
    dispatcher.add_handler(user_name_handler)

    #User sex
    user_sex_handler = CommandHandler("user_sex", user_sex)
    dispatcher.add_handler(user_sex_handler)
    
    #User race
    user_race_handler = CommandHandler("user_race", user_race)
    dispatcher.add_handler(user_race_handler)

    #User starter pack
    user_spack_handler = CommandHandler("user_spack", starter_pack)
    dispatcher.add_handler(user_spack_handler)

    #User registry finish
    userReg_fin_handler = CommandHandler("finish_registration", user_finish)
    dispatcher.add_handler(userReg_fin_handler)

    branch_user_handler = ConversationHandler(
        entry_points = [CallbackQueryHandler(user_name, 'newGame')], 
        states={
            STEP_TWO:   [MessageHandler(Filters.text & (~ Filters.command), user_sex)],
            STEP_THREE: [CallbackQueryHandler(user_race, ('male' or 'female'))],
            STEP_FOUR:  [CallbackQueryHandler(starter_pack, 'humanRace')]
        }, 
        fallbacks=[CommandHandler("menu", back_main_menu)]
    )

    dispatcher.add_handler(branch_user_handler)


amenu_handler = CommandHandler("amenu", amenu)
dispatcher.add_handler(amenu_handler)

start_game_handler = CommandHandler("start_game", start_game)
dispatcher.add_handler(start_game_handler)

back_main_menu_handler = CommandHandler("menu", back_main_menu)
dispatcher.add_handler(back_main_menu_handler)
#Buttons
dispatcher.add_handler(CallbackQueryHandler(button))

#echo
#echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
#dispatcher.add_handler(echo_handler)

#Turn off bot
stop_handler = CommandHandler('stop', stop)
dispatcher.add_handler(stop_handler)

#User info

dispatcher.add_handler(CommandHandler("say", player_reply))

updater.start_polling()
updater.idle()

if __name__ == "__reUser__":
    regUser()